In [ ]:
def get_top_sentences(user_input, model, tokenizer):
    bot_input_ids = News_to_input(user_input, openapi_key, tokenizer)
    
    chat_history_ids = summary(args, bot_input_ids, 0, '', None, model)
    pred_lst = list(chat_history_ids[0])
    final_text = []
    for p in pred_lst:
        if(p < len(user_input.split('. ')) and len(user_input.split('. ')[p]) > 10):
            final_text.append((p, user_input.split('. ')[p]))
    return final_text

In [179]:
"""
    Main training workflow
"""
from __future__ import division

import argparse
import glob
import os
import random
import signal
import time
import numpy as np

import torch
from pytorch_pretrained_bert import BertConfig


import distributed
from models import data_loader, model_builder
from models.data_loader import load_dataset
from models.model_builder import Summarizer
from tensorboardX import SummaryWriter
from models.reporter import ReportMgr
from models.stats import Statistics
from others.logging import logger
# from models.trainer import build_trainer
# build_trainer의 dependency package pyrouge.utils가 import되지 않아 직접 셀에 삽입
from others.logging import logger, init_logger
import easydict

args = easydict.EasyDict({
    "encoder":'classifier',
    "mode":'summary',
    "bert_data_path":'../bert_sample/korean',
    "model_path":'../models/bert_classifier',
    "bert_model":'../001_bert_morp_pytorch',
    "result_path":'../results/korean',
    "temp_dir":'.',
    "bert_config_path":'../001_bert_morp_pytorch/bert_config.json',
    "batch_size":1000,
    "use_interval":True,
    "hidden_size":128,
    "ff_size":512,
    "heads":4,
    "inter_layers":2,
    "rnn_size":512,
    "param_init":0,
    "param_init_glorot":True,
    "dropout":0.1,
    "optim":'adam',
    "lr":2e-3,
    "report_every":1,
    "save_checkpoint_steps":5,
    "block_trigram":True,
    "recall_eval":False,
    
    "accum_count":1,
    "world_size":1,
    "visible_gpus":'0',
    "gpu_ranks":'0',
    "log_file":'../logs/bert_classifier',
    "test_from":'../models/bert_classifier2/model_step_35000.pt'
})


def build_trainer(args, device_id, model,
                  optim):
    """
    Simplify `Trainer` creation based on user `opt`s*
    Args:
        opt (:obj:`Namespace`): user options (usually from argument parsing)
        model (:obj:`onmt.models.NMTModel`): the model to train
        fields (dict): dict of fields
        optim (:obj:`onmt.utils.Optimizer`): optimizer used during training
        data_type (str): string describing the type of data
            e.g. "text", "img", "audio"
        model_saver(:obj:`onmt.models.ModelSaverBase`): the utility object
            used to save the model
    """
    device = "cpu" if args.visible_gpus == '-1' else "cuda"


    grad_accum_count = args.accum_count
    n_gpu = args.world_size

    if device_id >= 0:
        gpu_rank = int(args.gpu_ranks[device_id])
    else:
        gpu_rank = 0
        n_gpu = 0

    #print('gpu_rank %d' % gpu_rank)

    tensorboard_log_dir = args.model_path

    writer = SummaryWriter(tensorboard_log_dir, comment="Unmt")

    report_manager = ReportMgr(args.report_every, start_time=-1, tensorboard_writer=writer)

    trainer = Trainer(args, model, optim, grad_accum_count, n_gpu, gpu_rank, report_manager)

    # print(tr)
    if (model):
        n_params = _tally_parameters(model)
        #logger.info('* number of parameters: %d' % n_params)

    return trainer

class Trainer(object):
    """
    Class that controls the training process.

    Args:
            model(:py:class:`onmt.models.model.NMTModel`): translation model
                to train
            train_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            valid_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            optim(:obj:`onmt.utils.optimizers.Optimizer`):
               the optimizer responsible for update
            trunc_size(int): length of truncated back propagation through time
            shard_size(int): compute loss in shards of this size for efficiency
            data_type(string): type of the source input: [text|img|audio]
            norm_method(string): normalization methods: [sents|tokens]
            grad_accum_count(int): accumulate gradients this many times.
            report_manager(:obj:`onmt.utils.ReportMgrBase`):
                the object that creates reports, or None
            model_saver(:obj:`onmt.models.ModelSaverBase`): the saver is
                used to save a checkpoint.
                Thus nothing will be saved if this parameter is None
    """

    def __init__(self,  args, model,  optim,
                  grad_accum_count=1, n_gpu=1, gpu_rank=1,
                  report_manager=None):
        # Basic attributes.
        self.args = args
        self.save_checkpoint_steps = args.save_checkpoint_steps
        self.model = model
        self.optim = optim
        self.grad_accum_count = grad_accum_count
        self.n_gpu = n_gpu
        self.gpu_rank = gpu_rank
        self.report_manager = report_manager

        self.loss = torch.nn.BCELoss(reduction='none')
        assert grad_accum_count > 0
        # Set model in training mode.
        if (model):
            self.model.train()

    def summary(self, test_iter, step, cal_lead=False, cal_oracle=False):
        """ Validate model.
            valid_iter: validate data iterator
        Returns:
            :obj:`nmt.Statistics`: validation loss statistics
        """
        # Set model in validating mode.
        def _get_ngrams(n, text):
            ngram_set = set()
            text_length = len(text)
            max_index_ngram_start = text_length - n
            for i in range(max_index_ngram_start + 1):
                ngram_set.add(tuple(text[i:i + n]))
            return ngram_set

        def _block_tri(c, p):
            tri_c = _get_ngrams(3, c.split())
            for s in p:
                tri_s = _get_ngrams(3, s.split())
                if len(tri_c.intersection(tri_s))>0:
                    return True
            return False

        if (not cal_lead and not cal_oracle):
            self.model.eval()
        stats = Statistics()

        
        with torch.no_grad():
            for batch in test_iter:
                src = batch.src
                labels = batch.labels
                segs = batch.segs
                clss = batch.clss
                mask = batch.mask
                mask_cls = batch.mask_cls


                gold = []
                pred = []

                if (cal_lead):
                    selected_ids = [list(range(batch.clss.size(1)))] * batch.batch_size
                elif (cal_oracle):
                    selected_ids = [[j for j in range(batch.clss.size(1)) if labels[i][j] == 1] for i in
                                    range(batch.batch_size)]
                else:
                    sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

                    # loss = self.loss(sent_scores, labels.float())
                    # loss = (loss * mask.float()).sum()
                    # batch_stats = Statistics(float(loss.cpu().data.numpy()), len(labels))
                    # stats.update(batch_stats)

                    sent_scores = sent_scores + mask.float()
                    sent_scores = sent_scores.cpu().data.numpy()
                    selected_ids = np.argsort(-sent_scores, 1)
                # selected_ids = np.sort(selected_ids,1)
                

        return selected_ids


    def _gradient_accumulation(self, true_batchs, normalization, total_stats,
                               report_stats):
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            if self.grad_accum_count == 1:
                self.model.zero_grad()

            src = batch.src
            labels = batch.labels
            segs = batch.segs
            clss = batch.clss
            mask = batch.mask
            mask_cls = batch.mask_cls

            sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

            loss = self.loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()
            (loss/loss.numel()).backward()
            # loss.div(float(normalization)).backward()

            batch_stats = Statistics(float(loss.cpu().data.numpy()), normalization)


            total_stats.update(batch_stats)
            report_stats.update(batch_stats)

            # 4. Update the parameters and statistics.
            if self.grad_accum_count == 1:
                # Multi GPU gradient gather
                if self.n_gpu > 1:
                    grads = [p.grad.data for p in self.model.parameters()
                             if p.requires_grad
                             and p.grad is not None]
                    distributed.all_reduce_and_rescale_tensors(
                        grads, float(1))
                self.optim.step()

        # in case of multi step gradient accumulation,
        # update only after accum batches
        if self.grad_accum_count > 1:
            if self.n_gpu > 1:
                grads = [p.grad.data for p in self.model.parameters()
                         if p.requires_grad
                         and p.grad is not None]
                distributed.all_reduce_and_rescale_tensors(
                    grads, float(1))
            self.optim.step()

    def _save(self, step):
        real_model = self.model
        # real_generator = (self.generator.module
        #                   if isinstance(self.generator, torch.nn.DataParallel)
        #                   else self.generator)

        model_state_dict = real_model.state_dict()
        # generator_state_dict = real_generator.state_dict()
        checkpoint = {
            'model': model_state_dict,
            # 'generator': generator_state_dict,
            'opt': self.args,
            'optim': self.optim,
        }
        checkpoint_path = os.path.join(self.args.model_path, 'model_step_%d.pt' % step)
        #logger.info("Saving checkpoint %s" % checkpoint_path)
        # checkpoint_path = '%s_step_%d.pt' % (FLAGS.model_path, step)
        if (not os.path.exists(checkpoint_path)):
            torch.save(checkpoint, checkpoint_path)
            return checkpoint, checkpoint_path

    def _start_report_manager(self, start_time=None):
        """
        Simple function to start report manager (if any)
        """
        if self.report_manager is not None:
            if start_time is None:
                self.report_manager.start()
            else:
                self.report_manager.start_time = start_time

    def _maybe_gather_stats(self, stat):
        """
        Gather statistics in multi-processes cases

        Args:
            stat(:obj:onmt.utils.Statistics): a Statistics object to gather
                or None (it returns None in this case)

        Returns:
            stat: the updated (or unchanged) stat object
        """
        if stat is not None and self.n_gpu > 1:
            return Statistics.all_gather_stats(stat)
        return stat

    def _maybe_report_training(self, step, num_steps, learning_rate,
                               report_stats):
        """
        Simple function to report training stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_training` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_training(
                step, num_steps, learning_rate, report_stats,
                multigpu=self.n_gpu > 1)

    def _report_step(self, learning_rate, step, train_stats=None,
                     valid_stats=None):
        """
        Simple function to report stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_step` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_step(
                learning_rate, step, train_stats=train_stats,
                valid_stats=valid_stats)

    def _maybe_save(self, step):
        """
        Save the model if a model saver is set
        """
        if self.model_saver is not None:
            self.model_saver.maybe_save(step)

            
def summary(args, b_list, device_id, pt, step, model):
    device_id = 0
    device = "cpu" if args.visible_gpus == '-1' else "cuda"
    if (pt != ''):
        test_from = pt
    else:
        test_from = args.test_from
    
    opt = vars(checkpoint['opt'])
    for k in opt.keys():
        if (k in model_flags):
            setattr(args, k, opt[k])
    #print(args)

    #model.load_cp(checkpoint)
    #model.eval()

    test_iter =data_loader.Dataloader(args, _lazy_dataset_loader(b_list),
                                  args.batch_size, device,
                                  shuffle=False, is_test=True)
    trainer = build_trainer(args, device_id, model, None)
    result = trainer.summary(test_iter,step)
    return result

def _tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

args.gpu_ranks = [int(i) for i in args.gpu_ranks.split(',')]
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

init_logger(args.log_file)
device = "cpu" if args.visible_gpus == '-1' else "cuda"
device_id = 0 if device == "cuda" else -1
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']

import argparse
import json
import os
import time
import urllib3
from glob import glob
import collections
import six
import gc
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.utils import download as _download

def do_lang ( openapi_key, text ) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    requestJson = {  
        "argument": {
            "text": text,
            "analysis_code": "morp"
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  openapi_key},
        body=json.dumps(requestJson)
    )

    json_data = json.loads(response.data.decode('utf-8'))
    json_result = json_data["result"]
    
    if json_result == -1:
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason
    else:
        json_data = json.loads(response.data.decode('utf-8'))
    
        json_return_obj = json_data["return_object"]
        
        return_result = ""
        json_sentence = json_return_obj["sentence"]
        for json_morp in json_sentence:
            for morp in json_morp["morp"]:
                return_result = return_result+str(morp["lemma"])+"/"+str(morp["type"])+" "

        return return_result

def get_kobert_vocab(cachedir="./tmp/"):
    # Add BOS,EOS vocab
    tokenizer = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/tokenizers/kobert_news_wiki_ko_cased-1087f8699e.spiece",
        #"fname": "/opt/ml/SumAI/bertsum/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece",
        "chksum": "ae5711deb3",
    }
    
    vocab_info = tokenizer
    vocab_file = _download(
        #vocab_info["url"], vocab_info["fname"], vocab_info["chksum"], cachedir=cachedir
        vocab_info["url"], vocab_info["chksum"], cachedir=cachedir
    )

    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
        vocab_file[0], padding_token="[PAD]", bos_token="[BOS]", eos_token="[EOS]"
    )
    return vocab_b_obj

    
class BertData():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        #self.tokenizer = Tokenizer(vocab_file_path)
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [''.join(s) for s in src]


        idxs = [i for i, s in enumerate(src) if (len(s) > 0)]

        src = [src[i][:20000] for i in idxs]
        src = src[:10000]

        if (len(src) < 3):
            return None

        src_txt = [''.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = text.split(' ')
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        tgt_txt = None
        src_txt = [original_src_txt[i] for i in idxs]
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt
    
def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")
        
class Tokenizer(object):
    
    def __init__(self, vocab_file_path):
        self.vocab_file_path = vocab_file_path
        """Loads a vocabulary file into a dictionary."""
        vocab = collections.OrderedDict()
        index = 0
        with open(self.vocab_file_path, "r", encoding='utf-8') as reader:

            while True:
                token = convert_to_unicode(reader.readline())
                if not token:
                    break

          ### joonho.lim @ 2019-03-15
                if token.find('n_iters=') == 0 or token.find('max_length=') == 0 :

                    continue
                token = token.split('\t')[0].strip('_')

                token = token.strip()
                vocab[token] = index
                index += 1
        self.vocab = vocab
        
    def convert_tokens_to_ids(self, tokens):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            try:
                ids.append(self.vocab[token])
            except:
                ids.append(1)
        if len(ids) > 10000:
            raise ValueError(
                "Token indices sequence length is longer than the specified maximum "
                " sequence length for this BERT model ({} > {}). Running this"
                " sequence through BERT will result in indexing errors".format(len(ids), 10000)
            )
        return ids


def _lazy_dataset_loader(pt_file):
    dataset = pt_file    
    yield dataset
    
def News_to_input(text, openapi_key, tokenizer):
    newstemp = do_lang(openapi_key, text)
    news = newstemp.split(' ./SF ')[:-1]
    
    bertdata = BertData(tokenizer)
    sent_labels = [0] * len(news)
    tmp = bertdata.preprocess(news)
    if(not tmp): return None
    #print(tmp)
    b_data_dict = {"src":tmp[0],
               "tgt": [0],
               "labels":[0,0,0],
               "src_sent_labels":sent_labels,
               "segs":tmp[2],
               "clss":tmp[3],
               "src_txt":tmp[4],
               "tgt_txt":'hehe'}
    b_list = []
    b_list.append(b_data_dict) 
    return b_list

openapi_key = '9318dc23-24ac-4b59-a99e-a29ec170bf02'

/opt/conda/envs/python3.6/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [272]:
from tqdm import tqdm 

checkpoint = torch.load('../models/bert_classifier2/model_step_35000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
model.load_cp(checkpoint)
model.eval()  

def add_topk_to_df(df, model, tokenizer):
    
    start = time.time()
    topk = []
    for i,context in enumerate(tqdm(df['context2'])):
        context = eval(context)[:30]
        top = None
        if(len(context) > 3):
            top = get_top_sentences(' '.join(context), model, tokenizer)
        if(top):
            topk.append(top)
        else:
            topk.append('Hello world')
            
    df['topk'] = topk
    df = df.drop(df[df['topk'] == 'Hello world'].index)
    print(f"{time.time()-start:.4f} sec")
    return df

[2023-01-23 12:41:19,747 INFO] loading archive file ../001_bert_morp_pytorch
[2023-01-23 12:41:19,748 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [180]:
def add_context_to_df(df):
    contexts = []
    for i, cont in enumerate(tqdm(df['raw'])):
        if(cont):
            contexts.append(text_filter(cont))
        else:
            contexts.append('delete this')
    df['context2'] = contexts
    df = df.drop(df[df['context2'] == 'delete this'].index)
    return df

In [273]:

df = pd.read_csv("./preprocessed/kmeans_4/kmeans_4_부동산_20221201_20221203_crwal_news_context_raw.csv_pre.csv")

FileNotFoundError: [Errno 2] No such file or directory: './preprocessed/kmeans_4/kmeans_4_부동산_20221201_20221203_crwal_news_context_raw.csv_pre.csv'

In [274]:
df2 = add_topk_to_df(df, model, tokenizer)

 20%|██        | 18/88 [00:10<00:42,  1.66it/s]


KeyboardInterrupt: 

In [ ]:
for i,v in enumerate(eval(df2.context2[3])):
    print(i,v)

0 ‘세종 에버파크’가 지난달 25일부터 전국 만 19세 이상 수요자들로부터 선착순으로 발기인 접수를 받고 있다. 
1 특히 최근 오픈한 사업설명회장이 문전성시를 이루면서, 가파른 금리 인상 등으로 인해 침체된 부동산 시장과는 대비되는 모습이다. 
2 세종시 연기면 일원에 위치한 ‘세종 에버파크’는 지하 2층~지상 37층, 24개 동, 전용면적 74·84㎡, 총 3,012세대 규모의 대단지다. 
3 시공은 대형 건설사인 현대건설로 예정됐다. 
4 ‘세종 에버파크’는 그동안 세종시에서 보기 드물었던 3,012세대의 대단지로 공급된다는 소식에 수요자들의 꾸준한 관심이 이어졌다. 
5 지난 18일 사업설명회장 오픈 당시에는 입장하려는 방문객들의 긴 줄이 늘어져 있었으며, 세종시 무주택자 및 세종시 소재 직장인 무주택자들을 대상으로 진행한 발기인 가입 우선 접수도 성황리에 마감됐다. 
6 ‘세종 에버파크’는 장기일반 민간임대주택과 공공지원 민간임대주택 2가지 방식으로 조성된다. 
7 전체 3,000여 세대 중 먼저 50% 이하는 장기일반 민간임대주택으로 발기인을 모집한다. 
8 이어 촉진지구 지정 후 전체 세대의 50% 이상은 공공지원 민간임대주택으로 공급할 예정이다. 
9 장기일반 민간임대주택 발기인은 청약 통장 보유 여부에 관계없이 신청할 수 있고, 재당첨 제한도 받지 않는다. 
10 다주택자도 신청할 수 있으며, 지위권 보유 기간 동안 취득세·종합부동산세 등 세금에 대한 부담이 없다. 
11 초기 출자금 완납 후에는 주택 소유 여부와 관계없이 지위권 전매가 가능하다. 
12 또한 사업 승인 후에는 매매 전환 합의를 통해 10년간 확정 전세가로 임대료 인상 없이 내 집처럼 살다가 10년 후 분양 전환할 수 있다. 
13 ‘세종 에버파크’는 상품성도 뛰어나다. 
14 먼저 4Bay, 판상형 구조 중심으로 설계된다. 
15 판상형 아파트는 방과 거실이 전면에 배치돼, 집안 내부에 해가 잘 들어오고 환기가 우수하다. 
16 발코니 확장 시 서비스 면적이 크게 늘어나 보

In [ ]:
df2.topk[2]

[(17, ' 4Bay 구조도 강점이다. '),
 (16, ' 발코니 확장 시 서비스 면적이 크게 늘어나 보다 넓게 공간을 쓸 수 있다. '),
 (5,
  ' 지난 18일 사업설명회장 오픈 당시에는 입장하려는 방문객들의 긴 줄이 늘어져 있었으며, 세종시 무주택자 및 세종시 소재 직장인 무주택자들을 대상으로 진행한 발기인 가입 우선 접수도 성황리에 마감됐다. '),
 (8, ' 이어 촉진지구 지정 후 전체 세대의 50% 이상은 공공지원 민간임대주택으로 공급할 예정이다. '),
 (6, ' ‘세종 에버파크’는 장기일반 민간임대주택과 공공지원 민간임대주택 2가지 방식으로 조성된다. '),
 (10, ' 다주택자도 신청할 수 있으며, 지위권 보유 기간 동안 취득세·종합부동산세 등 세금에 대한 부담이 없다. '),
 (14, ' 먼저 4Bay, 판상형 구조 중심으로 설계된다. '),
 (13, ' ‘세종 에버파크’는 상품성도 뛰어나다. '),
 (15, ' 판상형 아파트는 방과 거실이 전면에 배치돼, 집안 내부에 해가 잘 들어오고 환기가 우수하다. '),
 (4,
  ' ‘세종 에버파크’는 그동안 세종시에서 보기 드물었던 3,012세대의 대단지로 공급된다는 소식에 수요자들의 꾸준한 관심이 이어졌다. '),
 (7, ' 전체 3,000여 세대 중 먼저 50% 이하는 장기일반 민간임대주택으로 발기인을 모집한다. '),
 (12,
  ' 또한 사업 승인 후에는 매매 전환 합의를 통해 10년간 확정 전세가로 임대료 인상 없이 내 집처럼 살다가 10년 후 분양 전환할 수 있다. '),
 (2,
  ' 세종시 연기면 일원에 위치한 ‘세종 에버파크’는 지하 2층~지상 37층, 24개 동, 전용면적 74·84㎡, 총 3,012세대 규모의 대단지다. '),
 (9, ' 장기일반 민간임대주택 발기인은 청약 통장 보유 여부에 관계없이 신청할 수 있고, 재당첨 제한도 받지 않는다. '),
 (3, ' 시공은 대형 건설사인 현대건설로 예정됐다. '),
 (11, ' 초기 출자금 완납 후에는

In [ ]:

    

print(get_top_sentences(user_input, model, tokenizer))

[(6, '\n가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다'), (4, '\n가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다'), (2, '\n앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다'), (3, '\n한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다'), (0, '\n일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다'), (1, '\n정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다'), (5, '\n그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다'), (7, '\n그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다')]


In [257]:
all_paths = [
 'bertopic(mpnet)',
 'bertopic(paraphrase)',
 'kmeans_12',
 'hdbscan',
 'kmeans_8',
 'bertopic(SR-BERT)',
 'kmeans_4']

In [253]:
import os
from os import listdir
from os.path import isfile, join, isdir
mypath = f'./{all_paths[6]}'
onlyfiles = [f for f in listdir(mypath+'/raw') if isfile(join(mypath+'/raw', f)) and f.split('.')[1] == 'csv']
onlyfiles

./bertopic(paraphrase)
['bertopic_윤석열_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221215_crwal_news_context_raw.csv', 'bertopic_윤석열_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_윤석열_20221201_20221215_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221215_crwal_news_context_raw.csv']
0 111


  0%|          | 0/111 [00:00<?, ?it/s]


NameError: name 'text_filter' is not defined

In [ ]:
for i,file in enumerate(onlyfiles):
    filepath = f'./preprocessed/{mypath}/{file}_pre.csv'
    df = pd.read_csv(filepath)
    print(i, len(df))
    new_df = add_topk_to_df(df, model, tokenizer)
    if not os.path.exists(f'./preprocessed2/{mypath}/'):
        os.makedirs(f'./preprocessed2/{mypath}/')
    new_df.to_csv(f'./preprocessed2/{mypath}/{file}_pre2.csv')

In [254]:
for i,file in enumerate(onlyfiles):
    filepath = f'{mypath}/raw/{onlyfiles[i]}'
    df = pd.read_csv(filepath)
    print(i, len(df))
    new_df = add_context_to_df(df)
    if not os.path.exists(f'./preprocessed/{mypath}/'):
        os.makedirs(f'./preprocessed/{mypath}/')
    new_df.to_csv(f'./preprocessed/{mypath}/{file}_pre.csv')


In [44]:
import re
from nltk import word_tokenize, sent_tokenize
def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+[^~][^%](?:\.[-_0-9a-z]+)+[^%]', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\[[^\]]*[◼|기자|뉴스|사진|자료|자료사진|출처|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|경제|한겨례|일보|미디어|데일리|한겨례|타임즈|위키트리|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지][^\]]*?\][ 제공]*')
    pattern_tag = re.compile(r'\<[^>][^가-힣 ]*?>')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_others.sub('', result)
    result = pattern_tag.sub('', result)

    # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()
    return result


def text_filter(text): # str -> 전처리 -> 문장 배열
    text = clean_byline(text)
    #exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣.,]+')
    #exclusions = exclude_pattern.findall(text)
    #result = exclude_pattern.sub(' ', text).strip()
    #spacing = Spacing()
    #kospacing_txt = spacing(result) 
    result = text.strip()
    sentences = sent_tokenize(result) 
    return sentences

In [45]:
import re
text = "신혜진 기자 어쩌구저쩌구 (2%~3&) 뉴스 제목 [차민수 기자] <tag>오늘</tag>부터 <p>마스크 해제</p> 책 이름 <안녕하세요> 저는 신혜진입니다 [연합뉴스] 제공"
#text = "여야는 앞서 종부세 중과세율(1.2~6.0%)을 적용하는 조정대상지역 2주택 이상 소유자에 대해 중과를 폐지하고 일반세율(0.5~2.7%)을 적용하는 방안 등을 담은 잠정 합의안을 도출한 상태다. 종부세법 개정안은 예산안이 처리되는대로 국회 본회의에 오를 것으로 보인다. "
#text = '''서대구 역세권 사업 ‘공공주도로 개발’ 뉴스9(대구) 입력 2022.12.01 (21:47) 수정 2022.12.01 (22:05)\n\n댓글\n\n좋아요\n\n공유하기 글씨 크게보기\n\n가\n\n글씨 작게보기\n\n고화질 표준화질 자동재생 키보드 컨트롤 안내 동영상영역 시작 동영상 시작 동영상영역 끝 동영상 고정 취소 이전기사 이전기사 다음기사 다음기사\n\n고화질 표준화질 자동재생 키보드 컨트롤 안내 동영상영역 시작 동영상 시작 동영상영역 끝 동영상설명 동영상 고정 취소\n\n[앵커]\n\n\n\n대구시가 서대구 역세권 개발 방식을 민·관 공동추진에서 공공주도로 바꾸기로 했습니다.'''

text_filter(text)

['신혜진 기자 어쩌구저쩌구 (2%~3&) 뉴스 제목 오늘부터 마스크 해제 책 이름 <안녕하세요> 저는 신혜진입니다']

In [19]:
#pattern_others2 = re.compile(r'\<[^가-힣][^>]*?\>')
#pattern_others2 = re.compile(r'\[.[^]]*?\]')
pattern_others = re.compile(r'\[[^\]]*[기자|사진|자료|자료사진|출처|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지][^\]]*?\]')

re.findall(pattern_others, text)

['[차민수 기자]', '[연합뉴스]']

In [34]:
i = 1

path = 'hdbscan'
onlyfiles = [f for f in listdir(path+'/raw') if isfile(join(path+'/raw', f)) and f.split('.')[1] == 'csv']


filepath = f'{path}/raw/{onlyfiles[i]}'
df = pd.read_csv(filepath)
raw = df.raw[6]

In [35]:
raw

'외지인 아파트 매입 두달새 5.8%P·1년새 17%P 하락\n\n서울거주자 춘천아파트 매입 한 달간 7건 사상 최저\n\n지방 부동산 활성화 위한 취득세 중과 해제 시급해져\n\n◇1일부터 시행되는 LTV·주택담보대출 완화 내용<제공=연합뉴스>\n\n수도권을 포함한 전국적인 부동산 규제지역 해제 이후 강원도내 아파트에 대한 외지인 투자가 급감한 것으로 나타났다.\n\n한국부동산원에 따르면 10월 기준 도내 아파트 매매거래 884건 중 외지인이 아파트를 매입한 거래는 245건, 27.7%로 조사됐다. 규제지역 해제 직전인 지난 8월에는 이 비율이 33.5%였다. 외지인 투자가 활발했던 지난해 10월 도내 아파트 매매거래 2,742건 중 외지인 매입 비율이 45.4%(1,246건)였던 점을 감안하면 1년 새 17.7%포인트나 하락한 셈이다.\n\n외지인들이 지난 8월까지 비규제지역인 강원도의 아파트를 활발하게 매입했지만 정부의 수도권 규제지역 해제가 본격화된 9월부터 그 수치가 뚝 떨어진 것.\n\n지역별로는 수도권과 가까운 춘천과 원주에서 외지인 매입 감소 폭이 컸다. 춘천지역 아파트를 서울 거주자가 산 경우는 지난해 10월 93건에서 올해 10월 7건으로 92.5% 급감했다. 춘천에서 월 단위로 서울 거주자 매입이 10건 이하를 기록한 것은 통계 작성이 시작된 2006년 1월 이후 사상 처음이다. 원주의 경우 같은 기간 서울 거주자 매입은 132건에서 10건으로 92.4%나 줄었다. 지난해 외지인 매입 비율이 전국 1위를 기록할 정도로 높았던 만큼 하락도 가팔랐다.\n\n토지 거래에서도 외지인 이탈이 두드러졌다. 지난 10월 강원지역 순수토지 거래(필지 수 기준) 중 외지인은 2,030건을 매입해 전년동월(3,532건) 대비 42.5% 감소했다. 같은 기간 도내 거주자가 매입한 건수는 15.4% 감소에 그쳤다.\n\n규제지역은 지난 9월부터 차례로 해제되는 추세로 최근에는 서울, 경기 일부를 제외한 모든 지역을 규제대상에서 해제했다. 더욱이 1일부터 15억원

In [247]:
text_filter(raw)

['수도권을 포함한 전국적인 부동산 규제지역 해제 이후 강원도내 아파트에 대한 외지인 투자가 급감한 것으로 나타났다. ',
 '한국부동산원에 따르면 10월 기준 도내 아파트 매매거래 884건 중 외지인이 아파트를 매입한 거래는 245건, 27.7%로 조사됐다. ',
 '규제지역 해제 직전인 지난 8월에는 이 비율이 33.5%였다. ',
 '외지인 투자가 활발했던 지난해 10월 도내 아파트 매매거래 2,742건 중 외지인 매입 비율이 45.4%(1,246건)였던 점을 감안하면 1년 새 17.7%포인트나 하락한 셈이다. ',
 '지역별로는 수도권과 가까운 춘천과 원주에서 외지인 매입 감소 폭이 컸다. ',
 '춘천지역 아파트를 서울 거주자가 산 경우는 지난해 10월 93건에서 올해 10월 7건으로 92.5% 급감했다. ',
 '춘천에서 월 단위로 서울 거주자 매입이 10건 이하를 기록한 것은 통계 작성이 시작된 2006년 1월 이후 사상 처음이다. ',
 '원주의 경우 같은 기간 서울 거주자 매입은 132건에서 10건으로 92.4%나 줄었다. ',
 '지난해 외지인 매입 비율이 전국 1위를 기록할 정도로 높았던 만큼 하락도 가팔랐다. ',
 '토지 거래에서도 외지인 이탈이 두드러졌다. ',
 '지난 10월 강원지역 순수토지 거래(필지 수 기준) 중 외지인은 2,030건을 매입해 전년동월(3,532건) 대비 42.5% 감소했다. ',
 '같은 기간 도내 거주자가 매입한 건수는 15.4% 감소에 그쳤다. ',
 '규제지역은 지난 9월부터 차례로 해제되는 추세로 최근에는 서울, 경기 일부를 제외한 모든 지역을 규제대상에서 해제했다. ',
 '더욱이 1일부터 15억원 초과 아파트에도 주택담보대출이 허용되는 등 추가 완화 방안이 본격화되면서 도내 외지인 매입 규모는 더욱 줄어들 전망이다. ',
 '이처럼 외지인들의 도내 아파트와 토지 매입이 줄어들 것으로 예상되자 지방 부동산의 취득세 중과 해제 등을 요구하는 목소리가 높아지고 있다. ',
 '강문식 한국공인중개사협회 춘천

In [165]:
text_filter(raw)

['페이지\n\n윤 대통령, 제1차 국정과제 점검회의 다주택자 세 중과 해제 검토 주택공급, 공공·민간 믹스…이달말 공공주택 50만가구 사전청약\n\n윤석열 대통령은 15일 국정과제 점검회의에서 규제 완화와 다주택자 세 중과 해제 검토,주택공급, 공공·민간 믹스 등 3대개혁 청사진을 제시했다.',
 '윤 대통령은 이날 오후 2시부터 청와대 영빈관에서 열린 제1차 국정과제 점검회의에서 국민 패널 2명으로부터 ‘내 집 마련’과 ‘부동산 경기 활성화’ 방안과 관련한 질문을 받고 부동산 시장 연착륙을 위한 규제 완화 필요성을 강조했다.',
 '전 정부에서 강화된 부동산 규제를 일제히 풀 경우 부작용을 우려해 그간 속도 조절을 해왔지만, 고금리 등으로 상황이 급변한 시장 대응을 위해 다시 규제 완화 고삐를 죄겠다고 했다.이같은 발언은 규제완화를 통한 부동산 시장을 활성화시키겠다는 것으로 풀이된다.',
 '불합리한 복합 규제 때문…규제 완화 ‘속도’\n\n윤 대통령은 “제가 정부를 맡기 전까지 공급 측면과 수요 측면의 불합리한 복합 규제 때문에 집값이 너무 천정부지로 솟고 거래물량이 위축됐다”며 “시장 정상화가 중요하다고 해서 많은 규제를 풀고 정상화하려고 했는데, 고금리 상황 때문에 다시 부동산 가격이 하락하는 추세를 보였다”고 진단했다.',
 '그러면서 “(전 정부의) 잘못된 정책으로 인한 현상이라고 해도, 그것을 일시에 제거하다 보면 시장에 혼란이 일어나서 결국 국민에게 불편을 줄 수 있어서 시장 정상화의 속도를 조율해야 한다고 생각했다”며 수요 규제를 조금 더 빠른 속도로 풀어나가서 시장이 안정 찾는데 최선의 노력할 것이라고 강조했다.',
 '이 같은 발언은 최근 고금리와 집값 하락 우려로 거래가 메마르면서 곳곳에서 ‘경고음’이 터지는 데 따른 것으로 풀이된다.',
 '내놓은 매물이 나가지 않아 이사를 못가는 상황이 발생하거나, ‘영끌’ 매입자들의 금리 부담 급증, 집값 하락세에 따른 보유세 부담 호소, 빠른 월세화 현상 등이다.',
 '다주택자 세 중과 해제 검

In [16]:
text_filter(raw)

['강해인 기자  기자페이지 윤 대통령, 제1차 국정과제 점검회의 다주택자 세 중과 해제 검토 주택공급, 공공 민간 믹스 이달말 공공주택 50만가구 사전청약 윤석열 대통령은 15일 국정과제 점검회의에서 규제 완화와 다주택자 세 중과 해제 검토,주택공급, 공공 민간 믹스 등 3대개혁 청사진을 제시했다.',
 '윤 대통령은 이날 오후 2시부터 청와대 영빈관에서 열린 제1차 국정과제 점검회의에서 국민 패널 2명으로부터  내 집 마련 과  부동산 경기 활성화  방안과 관련한 질문을 받고 부동산 시장 연착륙을 위한 규제 완화 필요성을 강조했다.',
 '전 정부에서 강화된 부동산 규제를 일제히 풀 경우 부작용을 우려해 그간 속도 조절을 해왔지만, 고금리 등으로 상황이 급변한 시장 대응을 위해 다시 규제 완화 고삐를 죄겠다고 했다.이같은 발언은 규제완화를 통한 부동산 시장을 활성화시키겠다는 것으로 풀이된다.',
 '불합리한 복합 규제 때문 규제 완화  속도 윤 대통령은  제가 정부를 맡기 전까지 공급 측면과 수요 측면의 불합리한 복합 규제 때문에 집값이 너무 천정부지로 솟고 거래물량이 위축됐다 며  시장 정상화가 중요하다고 해서 많은 규제를 풀고 정상화하려고 했는데, 고금리 상황 때문에 다시 부동산 가격이 하락하는 추세를 보였다 고 진단했다.',
 '그러면서  전 정부의  잘못된 정책으로 인한 현상이라고 해도, 그것을 일시에 제거하다 보면 시장에 혼란이 일어나서 결국 국민에게 불편을 줄 수 있어서 시장 정상화의 속도를 조율해야 한다고 생각했다 며 수요 규제를 조금 더 빠른 속도로 풀어나가서 시장이 안정 찾는데 최선의 노력할 것이라고 강조했다.',
 '이 같은 발언은 최근 고금리와 집값 하락 우려로 거래가 메마르면서 곳곳에서  경고음 이 터지는 데 따른 것으로 풀이된다.',
 '내놓은 매물이 나가지 않아 이사를 못가는 상황이 발생하거나,  영끌  매입자들의 금리 부담 급증, 집값 하락세에 따른 보유세 부담 호소, 빠른 월세화 현상 등이다.',
 '다주택자 세 중과 해제 검토